In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import mne
import pywt

mne.set_log_level("WARNING")

# =========================================
# 0) Paths + constants
# =========================================

BASE_DIR = Path(r"C:\SeniorProject_Data")

RUNS_CSV = Path(r"C:\SeniorProject\Good_Data\seizure_runs_details.csv")

OUT_FEAT_CSV = Path(r"C:\SeniorProject\Good_Data\MOV_Good_Runs.csv")

WINDOW_SEC   = 2.0         
PREICTAL_SEC = 15 * 60     

print("BASE_DIR:", BASE_DIR)
print("RUNS_CSV exists:", RUNS_CSV.exists())

runs_table = pd.read_csv(RUNS_CSV)
runs_table["subject"] = runs_table["subject"].astype(str).str.strip()   # مثل sub-001
runs_table["session"] = runs_table["session"].astype(str).str.strip()   # مثل ses-01
runs_table["run"]     = pd.to_numeric(runs_table["run"], errors="raise").astype(int)

print("Runs table shape:", runs_table.shape)
display(runs_table.head())



def build_state_mask_from_runs(raw, subject, session, run, runs_df, preictal_sec=PREICTAL_SEC):
    
    sf = raw.info["sfreq"]
    n_samples = raw.n_times
    mask = np.zeros(n_samples, dtype=np.int8)

    # نختار صفوف هذا المريض / السيشن / الرن من CSV
    sub_df = runs_df[
        (runs_df["subject"] == subject) &
        (runs_df["session"] == session) &
        (runs_df["run"] == int(run))
    ].copy()

    if sub_df.empty:
        print(f"ℹ No seizures in CSV for {subject} | {session} | run-{run:02d} → كل الماسك = 0")
        return mask

    sub_df["onset"]    = pd.to_numeric(sub_df["onset"], errors="coerce")
    sub_df["duration"] = pd.to_numeric(sub_df["duration"], errors="coerce")
    sub_df = sub_df.dropna(subset=["onset", "duration"])

    if sub_df.empty:
        print(f"ℹ Seizure rows for {subject} | {session} | run-{run:02d} have NaN onset/duration → كل الماسك = 0")
        return mask

    print(f"✅ Found {len(sub_df)} seizures in CSV for {subject} | {session} | run-{run:02d}")
    if "eventType" in sub_df.columns:
        print(sub_df[["onset", "duration", "eventType"]])
    else:
        print(sub_df[["onset", "duration"]])

    for _, row in sub_df.iterrows():
        onset_sec = float(row["onset"])
        dur_sec   = float(row["duration"])

        ictal_start = int(onset_sec * sf)
        ictal_end   = int((onset_sec + dur_sec) * sf)

        ictal_start = max(0, min(ictal_start, n_samples))
        ictal_end   = max(0, min(ictal_end,   n_samples))

        pre_start = max(0, ictal_start - int(preictal_sec * sf))

        # pre-ictal
        mask[pre_start:ictal_start] = 1
        # ictal
        mask[ictal_start:ictal_end] = 2

    n0 = int((mask == 0).sum())
    n1 = int((mask == 1).sum())
    n2 = int((mask == 2).sum())
    print(f"Samples per class: 0={n0} ({n0/sf:.1f}s), 1={n1} ({n1/sf:.1f}s), 2={n2/sf:.1f}s")

    return mask


# =========================================
# 2) ECG/EMG artefact helpers
# =========================================
def prepare_ref_for_regression(eeg_raw, ref_raw, ref_name="REF"):
    if ref_raw is None:
        print(f"[{ref_name}] No reference file.")
        return None

    sf_eeg = eeg_raw.info["sfreq"]
    sf_ref = ref_raw.info["sfreq"]

    ref = ref_raw.copy()
    if sf_ref != sf_eeg:
        print(f"[{ref_name}] Resampling reference from {sf_ref} Hz to {sf_eeg} Hz (ref only).")
        ref.resample(sfreq=sf_eeg, npad="auto")

    eeg_len = eeg_raw.n_times
    ref_data = ref.get_data()
    ref_len = ref_data.shape[1]

    if ref_len > eeg_len:
        ref_data = ref_data[:, :eeg_len]
    elif ref_len < eeg_len:
        pad = np.zeros((ref_data.shape[0], eeg_len - ref_len), dtype=ref_data.dtype)
        ref_data = np.concatenate([ref_data, pad], axis=1)

    if np.var(ref_data) < 1e-10:
        print(f"[{ref_name}] Reference almost flat → skip.")
        return None

    return ref_data


def regression_artifact_removal(eeg_raw, ref_raw, ref_name="REF"):
    ref_data = prepare_ref_for_regression(eeg_raw, ref_raw, ref_name=ref_name)
    if ref_data is None:
        print(f"[{ref_name}] Regression skipped.")
        return eeg_raw.copy()

    Y = eeg_raw.get_data()
    X_ref = ref_data

    X = np.vstack([X_ref, np.ones((1, X_ref.shape[1]))]).T
    Y_T = Y.T

    beta, _, _, _ = np.linalg.lstsq(X, Y_T, rcond=None)
    artefact = (X @ beta).T
    cleaned  = Y - artefact

    out = eeg_raw.copy()
    out._data = cleaned
    print(f"[{ref_name}] Regression artefact removal done.")
    return out


def has_ecg_artefact(eeg_raw, ecg_raw, corr_thresh=0.3):
    ref_data = prepare_ref_for_regression(eeg_raw, ecg_raw, ref_name="ECG")
    if ref_data is None:
        return False

    eeg_data = eeg_raw.get_data()
    ref_ch   = ref_data[0]

    corrs = []
    for ch_idx in range(eeg_data.shape[0]):
        sig = eeg_data[ch_idx]
        c = np.corrcoef(sig, ref_ch)[0, 1]
        if not np.isnan(c):
            corrs.append(abs(c))

    if not corrs:
        return False

    max_corr = max(corrs)
    print(f"[ECG] max |corr(EEG, ECG)| = {max_corr:.3f}")
    return max_corr >= corr_thresh


def has_emg_artefact(eeg_raw, high_freq_band=(40.0, 100.0), ratio_thresh=0.3):
    data = eeg_raw.get_data()
    sf   = eeg_raw.info["sfreq"]

    hf = mne.filter.filter_data(
        data, sf,
        l_freq=high_freq_band[0],
        h_freq=high_freq_band[1],
        verbose=False
    )

    hf_power    = np.mean(hf**2, axis=1)
    total_power = np.mean(data**2, axis=1) + 1e-12
    ratios = hf_power / total_power
    median_ratio = float(np.median(ratios))

    print(f"[EMG] median HF(40–100Hz)/total power = {median_ratio:.3f}")
    return median_ratio >= ratio_thresh


def wavelet_denoise_emg_raw(eeg_raw, wavelet="db4", level=2, thr_scale=0.5):
    data = eeg_raw.get_data()
    denoised = np.zeros_like(data)

    for ch in range(data.shape[0]):
        sig = data[ch, :]
        coeffs = pywt.wavedec(sig, wavelet, level=level)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745
        thr = thr_scale * sigma * np.sqrt(2 * np.log(len(sig)))

        new_coeffs = [coeffs[0]]
        for d in coeffs[1:]:
            new_d = pywt.threshold(d, thr, mode="soft")
            new_coeffs.append(new_d)

        recon = pywt.waverec(new_coeffs, wavelet)
        if len(recon) > len(sig):
            recon = recon[:len(sig)]
        elif len(recon) < len(sig):
            pad = np.zeros(len(sig) - len(recon))
            recon = np.concatenate([recon, pad])
        denoised[ch, :] = recon

    out = eeg_raw.copy()
    out._data = denoised
    print("[EMG] Wavelet denoising done.")
    return out



def window_and_label(eeg_raw, state_mask, win_sec=WINDOW_SEC):

    sf = eeg_raw.info["sfreq"]
    data = eeg_raw.get_data()
    n_ch, n_samples = data.shape

    win_samples = int(win_sec * sf)
    if n_samples < win_samples:
        print("⚠ Run too short.")
        return None, None

    mask = np.asarray(state_mask, dtype=np.int8)
    if mask.size < n_samples:
        pad = np.zeros(n_samples - mask.size, dtype=np.int8)
        mask = np.concatenate([mask, pad])
    elif mask.size > n_samples:
        mask = mask[:n_samples]

    starts = np.arange(0, n_samples - win_samples + 1, win_samples)

    X_list, y_list = [], []
    for s in starts:
        e = s + win_samples
        seg = data[:, s:e]
        seg_mask = mask[s:e]

        if seg_mask.size == 0:
            lab = 0
        else:
            if (seg_mask == 2).any():
                lab = 2
            elif (seg_mask == 1).any():
                lab = 1
            else:
                lab = 0

        X_list.append(seg)
        y_list.append(lab)

    if not X_list:
        return None, None

    X = np.stack(X_list, axis=0)          # (n_windows, n_channels, n_samples)
    y = np.array(y_list, dtype=np.int8)   # (n_windows,)

    print(f"[window_and_label] X shape = {X.shape}")
    print(f"[window_and_label] label counts = {dict(zip(*np.unique(y, return_counts=True)))}")
    return X, y


# =========================================
# 4) Feature helpers PER-CHANNEL
# =========================================
def compute_bandpowers_and_relative(x, sf):
    """
    - Absolute bandpowers:
        Delta (0.5–4), Theta (4–8), Alpha (8–12),
        Beta (12–30), Gamma (30–45)
    - Relative bandpowers = abs / sum(abs)
    """
    x = np.asarray(x, dtype=float)
    n = x.size

    freqs = np.fft.rfftfreq(n, d=1.0 / sf)
    fft_vals = np.fft.rfft(x)
    psd = (np.abs(fft_vals) ** 2) / n  # power spectrum

    def band_power(fmin, fmax):
        mask = (freqs >= fmin) & (freqs < fmax)
        if not np.any(mask):
            return 0.0
        return psd[mask].sum()

    bp_delta = band_power(0.5, 4.0)
    bp_theta = band_power(4.0, 8.0)
    bp_alpha = band_power(8.0, 12.0)
    bp_beta  = band_power(12.0, 30.0)
    bp_gamma = band_power(30.0, 45.0)

    total_bp = bp_delta + bp_theta + bp_alpha + bp_beta + bp_gamma + 1e-12

    return {
        "Delta Bandpower": bp_delta,
        "Theta Bandpower": bp_theta,
        "Alpha Bandpower": bp_alpha,
        "Beta Bandpower":  bp_beta,
        "Gamma Bandpower": bp_gamma,

        "Relative Delta Bandpower": bp_delta / total_bp,
        "Relative Theta Bandpower": bp_theta / total_bp,
        "Relative Alpha Bandpower": bp_alpha / total_bp,
        "Relative Beta Bandpower":  bp_beta  / total_bp,
        "Relative Gamma Bandpower": bp_gamma / total_bp,
    }


def compute_time_features(x):
    """
    - Interquartile Range
    - Median Absolute Deviation
    - Mean
    - Median
    - Variance
    - Entropy
    - Standard Deviation
    - Skewness
    - Kurtosis
    - Line Length
    - Hjorth Parameters: activity, mobility, complexity
    """
    x = np.asarray(x, dtype=float)

    mean = np.mean(x)
    median = np.median(x)
    var = np.var(x)
    std = np.sqrt(var + 1e-12)

    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3 - q1

    mad = np.median(np.abs(x - median))

    # Entropy
    if np.allclose(x, x[0]):
        entropy = 0.0
    else:
        hist, _ = np.histogram(x, bins=50, density=True)
        p = hist + 1e-12
        p = p / p.sum()
        entropy = -np.sum(p * np.log2(p))

    centered = x - mean
    m3 = np.mean(centered**3)
    m4 = np.mean(centered**4)
    skew = m3 / (std**3 + 1e-12)
    kurt = m4 / (std**4 + 1e-12)

    line_len = np.sum(np.abs(np.diff(x)))

    activity = var
    dx = np.diff(x)
    var_dx = np.var(dx)
    mobility = np.sqrt(var_dx / (var + 1e-12))

    ddx = np.diff(dx)
    var_ddx = np.var(ddx) if ddx.size > 0 else 0.0
    mobility_dx = np.sqrt(var_ddx / (var_dx + 1e-12)) if var_dx > 0 else 0.0
    complexity = mobility_dx / (mobility + 1e-12)

    return {
        "Interquartile Range": iqr,
        "Median Absolute Deviation": mad,
        "Mean": mean,
        "Median": median,
        "Variance": var,
        "Entropy": entropy,
        "Standard Deviation": std,
        "Skewness": skew,
        "Kurtosis": kurt,
        "Line Length": line_len,
        "Hjorth Activity": activity,
        "Hjorth Mobility": mobility,
        "Hjorth Complexity": complexity,
    }


def compute_features_for_window(seg, sf):

    n_ch, _ = seg.shape
    feat_dict = {}

    for ch in range(n_ch):
        x = seg[ch, :]

        bp_dict = compute_bandpowers_and_relative(x, sf)
        td_dict = compute_time_features(x)

        prefix = f"ch{ch+1}_"

        for k, v in {**bp_dict, **td_dict}.items():
            feat_dict[prefix + k] = v

    return feat_dict



def process_one_run(subject, session, run, runs_df):

    print("\n====================================")
    print(f"Processing {subject} | {session} | run-{run:02d}")
    print("====================================")

    run_base = f"{subject}_{session}_task-szMonitoring_run-{run:02d}"

    eeg_path = BASE_DIR / subject / session / "eeg" / f"{run_base}_eeg.edf"
    ecg_path = BASE_DIR / subject / session / "ecg" / f"{run_base}_ecg.edf"
    emg_path = BASE_DIR / subject / session / "emg" / f"{run_base}_emg.edf"

    print("EEG path:", eeg_path, "| exists:", eeg_path.exists())
    print("ECG path:", ecg_path, "| exists:", ecg_path.exists())
    print("EMG path:", emg_path, "| exists:", emg_path.exists())

    if not eeg_path.exists():
        print(f"❌ EEG file not found → SKIP this run.")
        return None

    try:
        eeg_raw = mne.io.read_raw_edf(eeg_path, preload=True, verbose="ERROR")
    except Exception as e:
        print(f"❌ Error loading EEG: {e}")
        return None

    ecg_raw = mne.io.read_raw_edf(ecg_path, preload=True, verbose="ERROR") if ecg_path.exists() else None
    emg_raw = mne.io.read_raw_edf(emg_path, preload=True, verbose="ERROR") if emg_path.exists() else None

    print("EEG sfreq:", eeg_raw.info["sfreq"], "| n_channels:", len(eeg_raw.ch_names))

    # 1) Notch 50 Hz
    eeg_notch = eeg_raw.copy()
    eeg_notch.notch_filter(freqs=[50.0], verbose=False)

    # 2) Band-pass 0.5–45 Hz
    eeg_bp = eeg_notch.copy()
    eeg_bp.filter(l_freq=0.5, h_freq=45.0, verbose=False)

    # 3) state mask من CSV
    state_mask = build_state_mask_from_runs(
        eeg_bp, subject=subject, session=session, run=run, runs_df=runs_df, preictal_sec=PREICTAL_SEC
    )

    # 4) ECG regression
    if ecg_raw is not None and has_ecg_artefact(eeg_bp, ecg_raw):
        eeg_ecg = regression_artifact_removal(eeg_bp, ecg_raw, ref_name="ECG")
    else:
        print("[ECG] No strong ECG artefact → skipping")
        eeg_ecg = eeg_bp.copy()

    # 5) EMG wavelet denoise
    if has_emg_artefact(eeg_ecg):
        eeg_clean = wavelet_denoise_emg_raw(eeg_ecg, wavelet="db4", level=2, thr_scale=0.5)
    else:
        print("[EMG] No strong EMG artefact → skipping")
        eeg_clean = eeg_ecg.copy()

    # 6) Windowing
    X, y = window_and_label(eeg_clean, state_mask, win_sec=WINDOW_SEC)
    if X is None or y is None:
        print("⚠ No windows for this run.")
        return None

    n_normal   = int(np.sum(y == 0))
    n_preictal = int(np.sum(y == 1))
    n_ictal    = int(np.sum(y == 2))

    print("\n[Window Counts]")
    print(f"\U0001F7E6  Normal: {n_normal} windows")
    print(f"\U0001F7E7  Pre-ictal: {n_preictal} windows")
    print(f"\U0001F7E5  Ictal: {n_ictal} windows")
    print(f"\n➡ Finished run with Total windows: {len(y)}\n")
    # =============================================

    sf = float(eeg_clean.info["sfreq"])
    n_windows = X.shape[0]
    n_ch = X.shape[1]
    print(f"Windows: {n_windows}, Channels: {n_ch}")

    rows = []
    for w in range(n_windows):
        seg = X[w, :, :]  # (n_ch, n_samples)
        feat_dict = compute_features_for_window(seg, sf)

        row = {
            "subject": subject,
            "run": int(run),
            "window_idx": int(w),
            "label": int(y[w]),
        }
        row.update(feat_dict)
        rows.append(row)

    df_run = pd.DataFrame(rows)
    print("Run features shape:", df_run.shape)
    return df_run



runs_table["sub_id"] = runs_table["subject"].str.extract(r"(\d+)").astype(int)

runs_unique = runs_table[["subject", "session", "run", "sub_id"]].drop_duplicates()

print("🔢 Total unique runs in CSV:", len(runs_unique))

batch1 = runs_unique.drop(columns=["sub_id"]).head(200).copy()

print("✅ Batch 1 unique runs (first 200):", len(batch1))
display(batch1.head())
first = True

for idx, row in batch1.iterrows():
    subject = str(row["subject"]).strip()
    session = str(row["session"]).strip()
    run     = int(row["run"])

    print(f"\n➡ Running {subject} | {session} | run-{run:02d}")
    df_run = process_one_run(subject, session, run, runs_df=runs_table)

    if df_run is not None and not df_run.empty:
        if first:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="w")
            first = False
        else:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="a", header=False)

print("\n🎉 DONE! Saved Batch 1 per-window EEG features to:")
print(OUT_FEAT_CSV)

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import mne
import pywt

mne.set_log_level("WARNING")

# =========================================
# 0) Paths + constants
# =========================================

BASE_DIR = Path(r"C:\SeniorProject_Data")

RUNS_CSV = Path(r"C:\SeniorProject\Good_Data\seizure_runs_details.csv")

OUT_FEAT_CSV = Path(r"C:\SeniorProject\Good_Data\MOV_Good_Runs_Batch201-395.csv")

WINDOW_SEC   = 2.0          
PREICTAL_SEC = 15 * 60    
print("BASE_DIR:", BASE_DIR)
print("RUNS_CSV exists:", RUNS_CSV.exists())

runs_table = pd.read_csv(RUNS_CSV)
runs_table["subject"] = runs_table["subject"].astype(str).str.strip()   # مثل sub-001
runs_table["session"] = runs_table["session"].astype(str).str.strip()   # مثل ses-01
runs_table["run"]     = pd.to_numeric(runs_table["run"], errors="raise").astype(int)

print("Runs table shape:", runs_table.shape)
display(runs_table.head())



def build_state_mask_from_runs(raw, subject, session, run, runs_df, preictal_sec=PREICTAL_SEC):
    sf = raw.info["sfreq"]
    n_samples = raw.n_times
    mask = np.zeros(n_samples, dtype=np.int8)

    sub_df = runs_df[
        (runs_df["subject"] == subject) &
        (runs_df["session"] == session) &
        (runs_df["run"] == int(run))
    ].copy()

    if sub_df.empty:
        return mask

    sub_df["onset"]    = pd.to_numeric(sub_df["onset"], errors="coerce")
    sub_df["duration"] = pd.to_numeric(sub_df["duration"], errors="coerce")
    sub_df = sub_df.dropna(subset=["onset", "duration"])

    if sub_df.empty:
        return mask

    for _, row in sub_df.iterrows():
        onset_sec = float(row["onset"])
        dur_sec   = float(row["duration"])

        ictal_start = int(onset_sec * sf)
        ictal_end   = int((onset_sec + dur_sec) * sf)

        ictal_start = max(0, min(ictal_start, n_samples))
        ictal_end   = max(0, min(ictal_end,   n_samples))

        pre_start = max(0, ictal_start - int(preictal_sec * sf))

        mask[pre_start:ictal_start] = 1
        mask[ictal_start:ictal_end] = 2

    return mask



# =========================================
# 2) ECG/EMG artefact helpers
# =========================================
def prepare_ref_for_regression(eeg_raw, ref_raw, ref_name="REF"):
    if ref_raw is None:
        print(f"[{ref_name}] No reference file.")
        return None

    sf_eeg = eeg_raw.info["sfreq"]
    sf_ref = ref_raw.info["sfreq"]

    ref = ref_raw.copy()
    if sf_ref != sf_eeg:
        print(f"[{ref_name}] Resampling reference from {sf_ref} Hz to {sf_eeg} Hz (ref only).")
        ref.resample(sfreq=sf_eeg, npad="auto")

    eeg_len = eeg_raw.n_times
    ref_data = ref.get_data()
    ref_len = ref_data.shape[1]

    if ref_len > eeg_len:
        ref_data = ref_data[:, :eeg_len]
    elif ref_len < eeg_len:
        pad = np.zeros((ref_data.shape[0], eeg_len - ref_len), dtype=ref_data.dtype)
        ref_data = np.concatenate([ref_data, pad], axis=1)

    if np.var(ref_data) < 1e-10:
        print(f"[{ref_name}] Reference almost flat → skip.")
        return None

    return ref_data


def regression_artifact_removal(eeg_raw, ref_raw, ref_name="REF"):
    ref_data = prepare_ref_for_regression(eeg_raw, ref_raw, ref_name=ref_name)
    if ref_data is None:
        print(f"[{ref_name}] Regression skipped.")
        return eeg_raw.copy()

    Y = eeg_raw.get_data()
    X_ref = ref_data

    X = np.vstack([X_ref, np.ones((1, X_ref.shape[1]))]).T
    Y_T = Y.T

    beta, _, _, _ = np.linalg.lstsq(X, Y_T, rcond=None)
    artefact = (X @ beta).T
    cleaned  = Y - artefact

    out = eeg_raw.copy()
    out._data = cleaned
    print(f"[{ref_name}] Regression artefact removal done.")
    return out


def has_ecg_artefact(eeg_raw, ecg_raw, corr_thresh=0.3):
    ref_data = prepare_ref_for_regression(eeg_raw, ecg_raw, ref_name="ECG")
    if ref_data is None:
        return False

    eeg_data = eeg_raw.get_data()
    ref_ch   = ref_data[0]

    corrs = []
    for ch_idx in range(eeg_data.shape[0]):
        sig = eeg_data[ch_idx]
        c = np.corrcoef(sig, ref_ch)[0, 1]
        if not np.isnan(c):
            corrs.append(abs(c))

    if not corrs:
        return False

    max_corr = max(corrs)
    print(f"[ECG] max |corr(EEG, ECG)| = {max_corr:.3f}")
    return max_corr >= corr_thresh


def has_emg_artefact(eeg_raw, high_freq_band=(40.0, 100.0), ratio_thresh=0.3):
    data = eeg_raw.get_data()
    sf   = eeg_raw.info["sfreq"]

    hf = mne.filter.filter_data(
        data, sf,
        l_freq=high_freq_band[0],
        h_freq=high_freq_band[1],
        verbose=False
    )

    hf_power    = np.mean(hf**2, axis=1)
    total_power = np.mean(data**2, axis=1) + 1e-12
    ratios = hf_power / total_power
    median_ratio = float(np.median(ratios))

    print(f"[EMG] median HF(40–100Hz)/total power = {median_ratio:.3f}")
    return median_ratio >= ratio_thresh


def wavelet_denoise_emg_raw(eeg_raw, wavelet="db4", level=2, thr_scale=0.5):
    data = eeg_raw.get_data()
    denoised = np.zeros_like(data)

    for ch in range(data.shape[0]):
        sig = data[ch, :]
        coeffs = pywt.wavedec(sig, wavelet, level=level)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745
        thr = thr_scale * sigma * np.sqrt(2 * np.log(len(sig)))

        new_coeffs = [coeffs[0]]
        for d in coeffs[1:]:
            new_d = pywt.threshold(d, thr, mode="soft")
            new_coeffs.append(new_d)

        recon = pywt.waverec(new_coeffs, wavelet)
        if len(recon) > len(sig):
            recon = recon[:len(sig)]
        elif len(recon) < len(sig):
            pad = np.zeros(len(sig) - len(recon))
            recon = np.concatenate([recon, pad])
        denoised[ch, :] = recon

    out = eeg_raw.copy()
    out._data = denoised
    print("[EMG] Wavelet denoising done.")
    return out



def window_and_label(eeg_raw, state_mask, win_sec=WINDOW_SEC):
 
    sf = eeg_raw.info["sfreq"]
    data = eeg_raw.get_data()
    n_ch, n_samples = data.shape

    win_samples = int(win_sec * sf)
    if n_samples < win_samples:
        print("⚠ Run too short.")
        return None, None

    mask = np.asarray(state_mask, dtype=np.int8)
    if mask.size < n_samples:
        pad = np.zeros(n_samples - mask.size, dtype=np.int8)
        mask = np.concatenate([mask, pad])
    elif mask.size > n_samples:
        mask = mask[:n_samples]

    starts = np.arange(0, n_samples - win_samples + 1, win_samples)

    X_list, y_list = [], []
    for s in starts:
        e = s + win_samples
        seg = data[:, s:e]
        seg_mask = mask[s:e]

        if seg_mask.size == 0:
            lab = 0
        else:
            if (seg_mask == 2).any():
                lab = 2
            elif (seg_mask == 1).any():
                lab = 1
            else:
                lab = 0

        X_list.append(seg)
        y_list.append(lab)

    if not X_list:
        return None, None

    X = np.stack(X_list, axis=0)          # (n_windows, n_channels, n_samples)
    y = np.array(y_list, dtype=np.int8)   # (n_windows,)

    print(f"[window_and_label] X shape = {X.shape}")
    print(f"[window_and_label] label counts = {dict(zip(*np.unique(y, return_counts=True)))}")
    return X, y


# =========================================
# 4) Feature helpers PER-CHANNEL
# =========================================
def compute_bandpowers_and_relative(x, sf):
    """
    - Absolute bandpowers:
        Delta (0.5–4), Theta (4–8), Alpha (8–12),
        Beta (12–30), Gamma (30–45)
    - Relative bandpowers = abs / sum(abs)
    """
    x = np.asarray(x, dtype=float)
    n = x.size

    freqs = np.fft.rfftfreq(n, d=1.0 / sf)
    fft_vals = np.fft.rfft(x)
    psd = (np.abs(fft_vals) ** 2) / n  # power spectrum

    def band_power(fmin, fmax):
        mask = (freqs >= fmin) & (freqs < fmax)
        if not np.any(mask):
            return 0.0
        return psd[mask].sum()

    bp_delta = band_power(0.5, 4.0)
    bp_theta = band_power(4.0, 8.0)
    bp_alpha = band_power(8.0, 12.0)
    bp_beta  = band_power(12.0, 30.0)
    bp_gamma = band_power(30.0, 45.0)

    total_bp = bp_delta + bp_theta + bp_alpha + bp_beta + bp_gamma + 1e-12

    return {
        "Delta Bandpower": bp_delta,
        "Theta Bandpower": bp_theta,
        "Alpha Bandpower": bp_alpha,
        "Beta Bandpower":  bp_beta,
        "Gamma Bandpower": bp_gamma,

        "Relative Delta Bandpower": bp_delta / total_bp,
        "Relative Theta Bandpower": bp_theta / total_bp,
        "Relative Alpha Bandpower": bp_alpha / total_bp,
        "Relative Beta Bandpower":  bp_beta  / total_bp,
        "Relative Gamma Bandpower": bp_gamma / total_bp,
    }


def compute_time_features(x):
    """
    - Interquartile Range
    - Median Absolute Deviation
    - Mean
    - Median
    - Variance
    - Entropy
    - Standard Deviation
    - Skewness
    - Kurtosis
    - Line Length
    - Hjorth Parameters: activity, mobility, complexity
    """
    x = np.asarray(x, dtype=float)

    mean = np.mean(x)
    median = np.median(x)
    var = np.var(x)
    std = np.sqrt(var + 1e-12)

    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3 - q1

    mad = np.median(np.abs(x - median))

    # Entropy
    if np.allclose(x, x[0]):
        entropy = 0.0
    else:
        hist, _ = np.histogram(x, bins=50, density=True)
        p = hist + 1e-12
        p = p / p.sum()
        entropy = -np.sum(p * np.log2(p))

    centered = x - mean
    m3 = np.mean(centered**3)
    m4 = np.mean(centered**4)
    skew = m3 / (std**3 + 1e-12)
    kurt = m4 / (std**4 + 1e-12)

    line_len = np.sum(np.abs(np.diff(x)))

    activity = var
    dx = np.diff(x)
    var_dx = np.var(dx)
    mobility = np.sqrt(var_dx / (var + 1e-12))

    ddx = np.diff(dx)
    var_ddx = np.var(ddx) if ddx.size > 0 else 0.0
    mobility_dx = np.sqrt(var_ddx / (var_dx + 1e-12)) if var_dx > 0 else 0.0
    complexity = mobility_dx / (mobility + 1e-12)

    return {
        "Interquartile Range": iqr,
        "Median Absolute Deviation": mad,
        "Mean": mean,
        "Median": median,
        "Variance": var,
        "Entropy": entropy,
        "Standard Deviation": std,
        "Skewness": skew,
        "Kurtosis": kurt,
        "Line Length": line_len,
        "Hjorth Activity": activity,
        "Hjorth Mobility": mobility,
        "Hjorth Complexity": complexity,
    }


def compute_features_for_window(seg, sf):

    n_ch, _ = seg.shape
    feat_dict = {}

    for ch in range(n_ch):
        x = seg[ch, :]

        bp_dict = compute_bandpowers_and_relative(x, sf)
        td_dict = compute_time_features(x)

        prefix = f"ch{ch+1}_"

        for k, v in {**bp_dict, **td_dict}.items():
            feat_dict[prefix + k] = v

    return feat_dict


def process_one_run(subject, session, run, runs_df):

    print("\n====================================")
    print(f"Processing {subject} | {session} | run-{run:02d}")
    print("====================================")

    run_base = f"{subject}_{session}_task-szMonitoring_run-{run:02d}"

    eeg_path = BASE_DIR / subject / session / "eeg" / f"{run_base}_eeg.edf"
    ecg_path = BASE_DIR / subject / session / "ecg" / f"{run_base}_ecg.edf"
    emg_path = BASE_DIR / subject / session / "emg" / f"{run_base}_emg.edf"

    if not eeg_path.exists():
        return None

    try:
        eeg_raw = mne.io.read_raw_edf(eeg_path, preload=True, verbose=False)
    except:
        return None

    ecg_raw = mne.io.read_raw_edf(ecg_path, preload=True, verbose=False) if ecg_path.exists() else None
    emg_raw = mne.io.read_raw_edf(emg_path, preload=True, verbose=False) if emg_path.exists() else None

    # Notch + Bandpass without printing
    eeg_bp = eeg_raw.copy()
    eeg_bp.notch_filter(freqs=[50.0], verbose=False)
    eeg_bp.filter(l_freq=0.5, h_freq=45.0, verbose=False)

    # state mask ONLY (no prints)
    state_mask = build_state_mask_from_runs(
        eeg_bp, subject=subject, session=session, run=run, runs_df=runs_df, preictal_sec=PREICTAL_SEC
    )

    # ECG regression without prints
    if ecg_raw is not None and has_ecg_artefact(eeg_bp, ecg_raw):
        eeg_ecg = regression_artifact_removal(eeg_bp, ecg_raw, ref_name="ECG")
    else:
        eeg_ecg = eeg_bp.copy()

    # EMG wavelet denoise without prints
    if has_emg_artefact(eeg_ecg):
        eeg_clean = wavelet_denoise_emg_raw(eeg_ecg)
    else:
        eeg_clean = eeg_ecg.copy()

    # Windowing
    X, y = window_and_label(eeg_clean, state_mask, win_sec=WINDOW_SEC)
    if X is None or y is None:
        return None

    # ======= ONLY the output you want =======
    n_normal   = int(np.sum(y == 0))
    n_preictal = int(np.sum(y == 1))
    n_ictal    = int(np.sum(y == 2))

    print(f"🟦  Normal: {n_normal} windows")
    print(f"🟧  Pre-ictal: {n_preictal} windows")
    print(f"🟥  Ictal: {n_ictal} windows")
    print(f"\n➡ Finished run with Total windows: {len(y)}\n")
    # ========================================

    # Feature extraction (no prints)
    sf = float(eeg_clean.info["sfreq"])
    rows = []
    for w in range(X.shape[0]):
        seg = X[w, :, :]
        feat_dict = compute_features_for_window(seg, sf)
        row = {"subject": subject, "run": int(run), "window_idx": int(w), "label": int(y[w])}
        row.update(feat_dict)
        rows.append(row)

    return pd.DataFrame(rows)





runs_table["sub_id"] = runs_table["subject"].str.extract(r"(\d+)").astype(int)

runs_unique = runs_table[["subject", "session", "run", "sub_id"]].drop_duplicates()

print("🔢 Total unique runs in CSV:", len(runs_unique))

batch1 = runs_unique.drop(columns=["sub_id"]).iloc[200:395].copy()

print("✅ Batch 1 unique runs (first 200):", len(batch1))
display(batch1.head())

first = True

for idx, row in batch1.iterrows():
    subject = str(row["subject"]).strip()
    session = str(row["session"]).strip()
    run     = int(row["run"])

    print(f"\n➡ Running {subject} | {session} | run-{run:02d}")
    df_run = process_one_run(subject, session, run, runs_df=runs_table)

    if df_run is not None and not df_run.empty:
        if first:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="w")
            first = False
        else:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="a", header=False)

print("\n🎉 DONE! Saved Batch 1 per-window EEG features to:")
print(OUT_FEAT_CSV)

In [2]:
import pandas as pd


batch1 = r"C:\Users\LENOVO\Desktop\sen_proj\MOV_Good_Runs_Batch1-200.csv"  
batch2 = r"C:\Users\LENOVO\Desktop\sen_proj\MOV_Good_Runs_Batch201-395.csv"  

OUT = r"C:\Users\LENOVO\Desktop\sen_proj\eeg_MERGED.csv" 



df1 = pd.read_csv(batch1)
df2 = pd.read_csv(batch2)

print("Batch 1 shape:", df1.shape)
print("Batch 2 shape:", df2.shape)



df_all = pd.concat([df1, df2], axis=0, ignore_index=True)
print("After concat shape:", df_all.shape)



key_cols = ["subject", "run", "window_idx"]
if "session" in df_all.columns:
    key_cols = ["subject", "session", "run", "window_idx"]

before_drop = df_all.shape[0]
df_all = df_all.drop_duplicates(subset=key_cols, keep="first")
after_drop = df_all.shape[0]

print(f"Duplicates removed: {before_drop - after_drop}")
print("Shape after drop_duplicates:", df_all.shape)


df_all = df_all.sort_values(by=key_cols).reset_index(drop=True)



run_key = ["subject", "run"]
if "session" in df_all.columns:
    run_key = ["subject", "session", "run"]

unique_runs = df_all[run_key].drop_duplicates()
n_runs = unique_runs.shape[0]

print(f"\n📊 Number of unique runs in final file: {n_runs}")
print("First few runs:")
print(unique_runs.head())


df_all.to_csv(OUT, index=False, encoding="utf-8-sig")
print("\n✅ Saved merged file to:")
print(OUT)


Batch 1 shape: (3593729, 50)
Batch 2 shape: (1809830, 50)
After concat shape: (5403559, 50)
Duplicates removed: 0
Shape after drop_duplicates: (5403559, 50)

📊 Number of unique runs in final file: 392
First few runs:
       subject  run
0      sub-001    5
31255  sub-001    7
46014  sub-001    8
71753  sub-002    1
77443  sub-002    2

✅ Saved merged file to:
C:\Users\LENOVO\Desktop\sen_proj\eeg_MERGED.csv
